In [89]:
from helpers import cd_to_datetime, datetime_to_str

class NearEarthObject:
    """A near-Earth object (NEO).

    An NEO encapsulates semantic and physical parameters about the object, such
    as its primary designation (required, unique), IAU name (optional), diameter
    in kilometers (optional - sometimes unknown), and whether it's marked as
    potentially hazardous to Earth.

    A `NearEarthObject` also maintains a collection of its close approaches -
    initialized to an empty collection, but eventually populated in the
    `NEODatabase` constructor.
    """

    def __init__(self, designation, name=None, diameter=float('nan'), hazardous='N'):
        """Create a new `NearEarthObject`.
        
        :param info: 
        required unique designation (str), 
        optional name (str), diameter (float), hazardous (bool)
        """
        
        self.designation = str(designation)
        self.name = str(name).title()
        self.diameter = diameter
        self.hazardous = hazardous
        
        self.approaches = []

    @property
    def fullname(self):
        """Return a representation of the full name of this NEO."""
        
        if self.name is None:
            return self.designation      
        else:
            return self.designation + " (" + self.name + ")"
    
    @property
    def hazard(self):
        """Return 'is/is not' depending on boolean value of hazardous."""
        
        if self.hazardous == 'N':
            hazard = 'is not'
        else:
            hazard = 'is'
        return hazard

    def __str__(self):
        """Return `str(self)`."""
        
        formatted_string = "NEO {} has a diameter {} km and {} potentially hazardous.".format(
        self.fullname, self.diameter, self.hazard
        )

        return formatted_string

class CloseApproach:
    """A close approach to Earth by an NEO.

    A `CloseApproach` encapsulates information about the NEO's close approach to
    Earth, such as the date and time (in UTC) of closest approach, the nominal
    approach distance in astronomical units, and the relative approach velocity
    in kilometers per second.

    A `CloseApproach` also maintains a reference to its `NearEarthObject` -
    initally, this information (the NEO's primary designation) is saved in a
    private attribute, but the referenced NEO is eventually replaced in the
    `NEODatabase` constructor.
    """

    def __init__(self, neo, time, distance=float('nan'), velocity=float('nan')):
        """Create a new `CloseApproach`.

        :param info: 
        required unique designation (str), can be passed from the neo type'
        required date (str);
        optional: distance (float), velocity (bool), neo (neo).
        """

        
        self.neo = neo
        self._designation = neo.designation
        self.time = cd_to_datetime(time)
        
        self.distance = distance
        self.velocity = velocity

        
    @property
    def time_str(self):
        """Return a formatted representation of this `CloseApproach`'s approach time.

        The value in `self.time` should be a Python `datetime` object. While a
        `datetime` object has a string representation, the default representation
        includes seconds - significant figures that don't exist in our input
        data set.

        The `datetime_to_str` method converts a `datetime` object to a
        formatted string that can be used in human-readable representations and
        in serialization to CSV and JSON files.
        """

        return datetime_to_str(self.time)

    
    def __str__(self):
        """Return `str(self)`."""
        
        formatted_string = "- On {} NEO {} approaching with speed {} km/s and distance {} au.".format(
        self.time_str, self.neo.fullname, self.velocity, self.distance
        )
            
        return formatted_string

    def __repr__(self):
        """Return `repr(self)`, a computer-readable string representation of this object."""
        return (f"CloseApproach(time={self.time_str}, distance={self.distance:.2f}, "
                f"velocity={self.velocity:.2f}, neo={self.neo!r})")

In [90]:
neo = NearEarthObject(123, 'Barsik', 21.21, True)
cap = CloseApproach(neo, '1900-Dec-27 12:12', 10, 0.1)

In [91]:
print(neo)
print(cap)

NEO 123 (Barsik) has a diameter 21.21 km and is potentially hazardous.
- On 1900-12-27 12:12 NEO 123 (Barsik) approaching with speed 0.1 km/s and distance 10 au.


### Loading datasets with `extract.py`

In [82]:
"""Extract data on near-Earth objects and close approaches from CSV and JSON files.

The `load_neos` function extracts NEO data from a CSV file, formatted as
described in the project instructions, into a collection of `NearEarthObject`s.

The `load_approaches` function extracts close approach data from a JSON file,
formatted as described in the project instructions, into a collection of
`CloseApproach` objects.

The main module calls these functions with the arguments provided at the command
line, and uses the resulting collections to build an `NEODatabase`.

You'll edit this file in Task 2.
"""
import csv
import json

#from models import NearEarthObject, CloseApproach


def load_neos(neo_csv_path = 'data/neos.csv') -> list:
    """Read near-Earth object information from a CSV file.

    :param neo_csv_path: A path to a CSV file containing data about near-Earth objects.
    :return: A collection of `NearEarthObject`s.
    """
    # List below will store instanciated NEOs objects 
    # new_neo = NearEarthObject(designation, name=None, diameter=float('nan'), hazardous=False)
    neos = []
    with open(neo_csv_path, 'r') as file:
        reader = csv.DictReader(file)
        for line in reader:
            neos.append(NearEarthObject(designation=line['pdes'], 
                                        name=line['name'], 
                                        diameter=line['diameter'], 
                                        hazardous=['pha']
                                       )
                       )
    return neos


def load_approaches(cad_json_path = 'data/cad.json') -> dict:
    """Read close approach data from a JSON file.

    :param neo_csv_path: A path to a JSON file containing data about close approaches.
    :return: A collection of `CloseApproach`es.
    """
    with open(cad_json_path, 'r') as file:
        cads = json.load(file)
    return cads

In [255]:
nn = load_neos()
len(nn)

23967

In [266]:
cc = load_approaches()
l = (cc['data'])

In [273]:
l.sort()

In [274]:
db = {}
for i in range(len(l)):
    pdes = l[i][0]
    if pdes in db.keys():
        date = l[i][3]
        distance = l[i][4]
        speed = l[i][7]
        db[pdes].append([{date:(distance, speed)}])
    else:
        db[pdes] = [{date:(distance, speed)}]

len(db)  

23424

In [275]:
db['444004']

[{'2099-Oct-12 12:38': ('0.30452900915319', '9.8781760028839')},
 [{'1901-Feb-01 17:22': ('0.317306929483672', '9.16507707064606')}],
 [{'1901-Aug-12 07:35': ('0.156798611127023', '8.80258435360103')}],
 [{'1902-Aug-26 05:03': ('0.494881718216013', '21.6877854066479')}],
 [{'1911-Feb-15 13:50': ('0.0813604942612545', '9.50159008762572')}],
 [{'1911-Sep-13 09:08': ('0.354094373895403', '8.37255075007055')}],
 [{'1912-Aug-17 05:58': ('0.268028007389014', '15.7303551033059')}],
 [{'1921-Feb-09 03:14': ('0.131408474981846', '12.9303388733646')}],
 [{'1921-Nov-13 11:46': ('0.426269726255214', '5.9205771753889')}],
 [{'1922-Aug-12 03:12': ('0.0750339715595801', '11.2521050955162')}],
 [{'1931-Feb-02 12:27': ('0.328262751473962', '17.7023612299244')}],
 [{'1932-Jan-26 09:07': ('0.339250517451086', '8.98732758655057')}],
 [{'1932-Aug-11 02:44': ('0.131828297765824', '8.92571579340475')}],
 [{'1942-Feb-16 00:03': ('0.138559076182843', '9.08868432910519')}],
 [{'1942-Aug-31 23:40': ('0.312918075

In [259]:
# df2 = pd.DataFrame.from_dict(data = cc['data'])
# df2.columns=cc['fields']
# df2.loc[df2['des']=="433"].head(5)
# #len(df2)

### Encapsulate the data in a `NEODatabase`

In [ ]:
"""A database encapsulating collections of near-Earth objects and their close approaches.

A `NEODatabase` holds an interconnected data set of NEOs and close approaches.
It provides methods to fetch an NEO by primary designation or by name, as well
as a method to query the set of close approaches that match a collection of
user-specified criteria.

Under normal circumstances, the main module creates one NEODatabase from the
data on NEOs and close approaches extracted by `extract.load_neos` and
`extract.load_approaches`.

"""
#from extract import load_neos, load_approaches

class NEODatabase:
    """A database of near-Earth objects and their close approaches.

    A `NEODatabase` contains a collection of NEOs and a collection of close
    approaches. It additionally maintains a few auxiliary data structures to
    help fetch NEOs by primary designation or by name and to help speed up
    querying for close approaches that match criteria.
    """
    def __init__(self, neos, approaches):
        """Create a new `NEODatabase`.

        As a precondition, this constructor assumes that the collections of NEOs
        and close approaches haven't yet been linked - that is, the
        `.approaches` attribute of each `NearEarthObject` resolves to an empty
        collection, and the `.neo` attribute of each `CloseApproach` is None.

        However, each `CloseApproach` has an attribute (`._designation`) that
        matches the `.designation` attribute of the corresponding NEO. This
        constructor modifies the supplied NEOs and close approaches to link them
        together - after it's done, the `.approaches` attribute of each NEO has
        a collection of that NEO's close approaches, and the `.neo` attribute of
        each close approach references the appropriate NEO.

        :param neos: A collection of `NearEarthObject`s.
        :param approaches: A collection of `CloseApproach`es.
        """
        self._neos = neos
        self._approaches = approaches
        
        
        

        # TODO: What additional auxiliary data structures will be useful?
# dict? {k:j}
        # TODO: Link together the NEOs and their close approaches.

    def get_neo_by_designation(self, designation):
        """Find and return an NEO by its primary designation.

        If no match is found, return `None` instead.

        Each NEO in the data set has a unique primary designation, as a string.

        The matching is exact - check for spelling and capitalization if no
        match is found.

        :param designation: The primary designation of the NEO to search for.
        :return: The `NearEarthObject` with the desired primary designation, or `None`.
        """
        # TODO: Fetch an NEO by its primary designation.
        return None

    def get_neo_by_name(self, name):
        """Find and return an NEO by its name.

        If no match is found, return `None` instead.

        Not every NEO in the data set has a name. No NEOs are associated with
        the empty string nor with the `None` singleton.

        The matching is exact - check for spelling and capitalization if no
        match is found.

        :param name: The name, as a string, of the NEO to search for.
        :return: The `NearEarthObject` with the desired name, or `None`.
        """
        # TODO: Fetch an NEO by its name.
        return None

    def query(self, filters=()):
        """Query close approaches to generate those that match a collection of filters.

        This generates a stream of `CloseApproach` objects that match all of the
        provided filters.

        If no arguments are provided, generate all known close approaches.

        The `CloseApproach` objects are generated in internal order, which isn't
        guaranteed to be sorted meaninfully, although is often sorted by time.

        :param filters: A collection of filters capturing user-specified criteria.
        :return: A stream of matching `CloseApproach` objects.
        """
        # TODO: Generate `CloseApproach` objects that match all of the filters.
        for approach in self._approaches:
            yield approach


### Example of class methods to dynamically create new classes:

In [611]:
class Dog():

    def __init__(self, name:str, age:int, breed:str, weight:int):
        """Create a new dog"""
        self.breed = breed
        self.weight = weight
        self.name = name
        self.age = age
        
    def __gt__(self, other):
        return self.age > other.age
    
    def __str__(self):
        return str(self.name)
    
    def speak(self) -> None:
        """Make the dog bark"""
        print(f'{self.name} says, "woof"')
    @classmethod    
    def spawn(cls, name, mother, father):
        breed = mother.breed
        if mother.breed != father.breed:
            breed = f'{mother.breed}-{father.breed} Mix'
        weight = (mother.weight + father.weight) / (2 * 10)
        return cls(name, 0, breed, weight)

if __name__ == "__main__":
    sally = Dog('Sally', 6, 'chihuahua', 7)
    henry = Dog('Henry', 7, 'terrier', 15)
    trixy = Dog.spawn('Trixy', sally, henry)
    print(trixy.breed)

chihuahua-terrier Mix


In [612]:
trixy.weight

1.1

In [618]:
(9257+725) / 12

831.8333333333334

In [621]:
199 * 12


2388

In [622]:
18 + 16 + 5

39